# plot and check Marchenko Pastur for different $\gamma=\frac{N}{M}$
To make sure I understood this correctly, I have to make sure everything works empirically. This tool a while with several AIs... Anyway, bottom line is that things seem to work. And for a certain $\gamma=\frac{N}{M}$ we shuold expect $s_{max}=1+
\sqrt{\gamma}$

In [86]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
N = 256
gamma_values = [128, 256]
num_Ts = 200000 

def generate_complex_matrix(N, M):
    """Generate a normalized complex Gaussian random matrix."""
    sigma = np.sqrt(2) / 2
    real_part = np.random.normal(0, sigma, size=(N, M))
    imag_part = np.random.normal(0, sigma, size=(N, M))
    A = (real_part + 1j * imag_part) / np.sqrt(M)  
    return A

def marchenko_pastur_svd_pdf(x, gamma):
    x_plus = (1 + np.sqrt(gamma)) ** 2
    x_minus = (1 - np.sqrt(gamma)) ** 2
    density = np.zeros_like(x)
    valid = (x ** 2 >= x_minus) & (x ** 2 <= x_plus)
    epsilon = 1e-15  # Small value to avoid division by zero
    density[valid] = (1 / (gamma*np.pi * np.maximum(x[valid], epsilon))) * np.sqrt((x_plus - x[valid] ** 2) * (x[valid] ** 2 - x_minus))
    
    if gamma > 1:
        density *= gamma
        
        # Add point mass at zero
        if x[0] == 0:
            density[0] = (gamma - 1) / gamma

    
    return density

fig, axs = plt.subplots(1, len(gamma_values), figsize=(18, 5))

for ax, gamma in zip(axs, gamma_values):
    M = int(N / gamma)
    singular_values = []
    
    # Generate matrices and compute singular values
    for _ in range(num_Ts):
        A = generate_complex_matrix(N, M)
        svd_vals = np.linalg.svd(A, compute_uv=False)
        singular_values.extend(svd_vals)
    
    singular_values = np.array(singular_values)
    print(f'{gamma=}')
    print(f'min={singular_values.min():.2f}, max={singular_values.max():.2f}')
    
    # Plot histogram of singular values
    ax.hist(singular_values, bins=50, density=True, alpha=0.7, label="Empirical")
    
    # Plot theoretical Marchenko-Pastur distribution for singular values
    x_vals = np.linspace(0, max(2, (1 + np.sqrt(gamma)) + 0.5), 1000)
    mp_pdf = marchenko_pastur_svd_pdf(x_vals, gamma)
    ax.plot(x_vals, mp_pdf, 'r-', label="Theoretical")
    
    min_sv = np.abs(1 - np.sqrt(gamma))
    max_sv = (1 + np.sqrt(gamma))
    
    ax.axvline(min_sv, color='g', linestyle='--', label=f'Min SV: {min_sv:.2f}')
    ax.axvline(max_sv, color='b', linestyle='--', label=f'Max SV: {max_sv:.2f}')
    
    if gamma > 1:
        ax.plot([0], [(gamma - 1) / gamma], 'ro', markersize=10, label="Point mass at 0")
    
    ax.set_title(f"γ = {gamma}")
    ax.set_xlabel("Singular Value")
    ax.set_ylabel("Density")
    ax.legend()
    fig.show()

plt.tight_layout()

gamma=128
min=9.50, max=13.30
gamma=256
min=13.81, max=18.66


In [108]:
N = 256
M = 512
A = generate_complex_matrix(N, M)
v_in = 1/np.sqrt(M)*np.ones(M)
(np.abs(A@v_in)**2).sum()

0.45002438108227266

# Expected maximal SVD value for finite size N 
This might explain the N dependance. For larger matrices we have a larger chance of hitting a large SVD value. (which is bounded from above by 4 for $N\rightarrow\infty$)

In [13]:
import numpy as np
from scipy.integrate import quad
import matplotlib.pyplot as plt

def marchenko_pastur_pdf(x, q):
    """
    PDF of the Marchenko-Pastur distribution.
    """
    a = (1 - np.sqrt(q))**2
    b = (1 + np.sqrt(q))**2
    if a <= x <= b:
        return np.sqrt((b - x) * (x - a)) / (2 * np.pi * q * x)
    else:
        return 0

def marchenko_pastur_cdf(x, q):
    """
    CDF of the Marchenko-Pastur distribution, computed by integrating the PDF.
    """
    a = (1 - np.sqrt(q))**2
    if x < a:
        return 0
    b = (1 + np.sqrt(q))**2
    if x > b:
        return 1
    result, _ = quad(lambda t: marchenko_pastur_pdf(t, q), a, x)
    return result

def expected_maximum(N, q):
    """
    Compute the expected maximum for N samples from the Marchenko-Pastur distribution.
    """
    a = (1 - np.sqrt(q))**2
    b = (1 + np.sqrt(q))**2

    def integrand(x):
        f_x = marchenko_pastur_pdf(x, q)
        F_x = marchenko_pastur_cdf(x, q)
        return x * N * (F_x**(N-1)) * f_x

    result, _ = quad(integrand, a, b)
    return result

# Parameters
q = 0.5  # Ratio M/N
N_values = 2**np.linspace(1, 20, 20)

# Compute E[max_s] for each N
expected_max_values = [expected_maximum(N, q) for N in N_values]

# Plot
plt.figure(figsize=(8, 6))
plt.plot(N_values, expected_max_values, marker="o", label=f"Marchenko-Pastur (q={q})", color="blue")
plt.xscale("log")
plt.xlabel("Number of Samples (N) [log scale]")
plt.ylabel("Expected Maximum (E[max_s])")
plt.title("Expected Maximum vs. Number of Samples for Marchenko-Pastur")
plt.grid(visible=True, which="both", linestyle="--", linewidth=0.5)
plt.legend()
plt.show()

# Print results for reference
for N, value in zip(N_values, expected_max_values):
    print(f"N={N}, E[max_s]={value:.6f}")


C:\Users\owner\AppData\Local\Temp\ipykernel_71288\3005568292.py:41: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  result, _ = quad(integrand, a, b)
C:\Users\owner\AppData\Local\Temp\ipykernel_71288\3005568292.py:41: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  result, _ = quad(integrand, a, b)


N=2.0, E[max_s]=1.395469
N=4.0, E[max_s]=1.790171
N=8.0, E[max_s]=2.128743
N=16.0, E[max_s]=2.387216
N=32.0, E[max_s]=2.569668
N=64.0, E[max_s]=2.692417
N=128.0, E[max_s]=2.772717
N=256.0, E[max_s]=2.824415
N=512.0, E[max_s]=2.857396
N=1024.0, E[max_s]=2.878327
N=2048.0, E[max_s]=2.891571
N=4096.0, E[max_s]=2.899936
N=8192.0, E[max_s]=2.905214
N=16384.0, E[max_s]=2.908543
N=32768.0, E[max_s]=2.910644
N=65536.0, E[max_s]=2.911977
N=131072.0, E[max_s]=2.912855
N=262144.0, E[max_s]=2.913552
N=524288.0, E[max_s]=2.914527
N=1048576.0, E[max_s]=2.916910


# optimize SLM3 "analytically" with SVD on N//2 pixels  

## what does phase only do?
It seems thhat thie higest $s^2$ that I get for $N=256$ is $\approx1.95$, which is significantly less than $4$. And that having phase only reduces the energy transfer by $\approx0.7$.  I still get $1.6$ which is nice, but we need to cut this in half because we only half the matrix. We do get also 0.25 from the othe r half. TODO this. 

In [178]:
import numpy as np

N = 256
sig_for_gauss_iid = np.sqrt(2) / 2
T = 1 / np.sqrt(N) * np.random.normal(loc=0, scale=sig_for_gauss_iid, size=(N, N, 2)).view(np.complex128)[:, :, 0]

T_sub = T[N//2:, :N//2]
U, S, Vh = np.linalg.svd(T_sub)
max_energy_input = Vh[0].conj()  # Corresponding to the largest singular value
max_energy_input_phase_only = 1/np.sqrt(N/2)*np.ones_like(max_energy_input) * np.exp(1j*np.angle(max_energy_input))

max_energy_output = T_sub @ max_energy_input
max_energy_output_phase_only = T_sub @ max_energy_input_phase_only
energy = np.sum(np.abs(max_energy_output)**2)
energy_phase_only = np.sum(np.abs(max_energy_output_phase_only)**2)

input_energy = (np.abs(max_energy_input_phase_only)**2).sum()
print(f'{input_energy=}')
print("Maximized energy transfer:", energy)
print("Maximized energy transfer phase only:", energy_phase_only)
print("ratio:", energy_phase_only**2 / energy**2)


input_energy=1.0
Maximized energy transfer: 1.9917731807656383
Maximized energy transfer phase only: 1.6873136856520992
ratio: 0.7176486917929871


In [55]:
# (np.abs(max_energy_input)**2).sum()
(np.abs(max_energy_input_phase_only)**2).sum()

0.5

In [168]:
import numpy as np

N = 256
sig_for_gauss_iid = np.sqrt(2) / 2
T = 1 / np.sqrt(N) * np.random.normal(loc=0, scale=sig_for_gauss_iid, size=(N, N, 2)).view(np.complex128)[:, :, 0]

# in_indexes = np.index_exp[N//2:] # TODO: choose randomly N/2 numbers, and extract the relevant sub matrix
T_sub = T[N//2:, :N//2]  
U, S, Vh = np.linalg.svd(T_sub)
good_input = Vh[0].conj()  # Corresponding to the largest singular value
tot_good_input_phase_only = 1/np.sqrt(N)*np.ones(N, dtype=np.complex128)
tot_good_input_phase_only[N//2:] *= np.exp(1j*np.angle(good_input))

output = T @ tot_good_input_phase_only
full_energy = np.sum(np.abs(output)**2)
half_energy = np.sum(np.abs(output[:N//2])**2)

output_sub = T_sub @ tot_good_input_phase_only[N//2:]
energy_sub = np.sum(np.abs(output_sub)**2)

print(f"{half_energy=:.2f}")
print(f"{full_energy=:.2f}")
print(f"{energy_sub=:.2f}")

half_energy=0.57
full_energy=1.02
energy_sub=0.79
